# Experiment with a 5x7 font for matching the numbers in the dots

Gary Bishop July 2018

In [ ]:
# a font I found online
digits = np.array([
    0x3E, 0x41, 0x41, 0x41, 0x3E, # 0
    0x00, 0x42, 0x7F, 0x40, 0x00, # 1
    0x42, 0x61, 0x51, 0x49, 0x46, # 2
    # 0x21, 0x41, 0x45, 0x4B, 0x31, # 3
    # 0x41, 0x49, 0x49, 0x49, 0x36, # 3
    #0x22, 0x49, 0x49, 0x5d, 0x36, # 3
    #0x0, 0x49, 0x49, 0x49, 0x36, # 3
    #0x0, 0x49, 0x49, 0x5d, 0x36, # 3
    0x0, 0x6d, 0x6d, 0x6f, 0x3b, # 3
    0x18, 0x14, 0x12, 0x7F, 0x10, # 4
    0x27, 0x45, 0x45, 0x45, 0x39, # 5
    0x3C, 0x4A, 0x49, 0x49, 0x30, # 6
    0x01, 0x71, 0x09, 0x05, 0x03, # 7
    0x36, 0x49, 0x49, 0x49, 0x36, # 8
    0x06, 0x49, 0x49, 0x29, 0x1E, # 9
])
digits.shape = (10,5)
def toTemplate(d):
    t = np.zeros((9,7), dtype=np.uint8)
    for y in range(7):
        for x in range(5):
            if d[x] & (1<<y):
                t[y+1,x+1] = 255
    return 255 - t

def fromTemplate(t):
    d = np.zeros(5, dtype=int)
    for y in range(7):
        for x in range(5):
            if t[y+1,x+1] == 0:
                d[x] |= 1<<y
    return ', '.join([hex(v) for v in d])

print(toTemplate(digits[3]))
print(fromTemplate(toTemplate(digits[3])))

In [ ]:
s3 = '''
|       |
|  **** |
|    ** |
|  ***  |
|  **** |
|     * |
|  **** |
|  ***  |
|       |
'''
print(fromEdit(s3))
fromTemplate(fromEdit(s3))

In [ ]:
toEditFormat = """'''
{data}
'''
"""
def toEdit(t):
    lines = []
    for row in t:
        line = []
        for value in row:
            line.append(' ' if value else '*')
        lines.append('|' + ''.join(line) + '|')
    return toEditFormat.format(data='\n'.join(lines))

def fromEdit(s):
    lines = s.split('\n')[1:]
    t = np.zeros((9,7), dtype=np.uint8)
    for y, line in enumerate(lines[:-1]):
        for x, char in enumerate(line[1:-1]):
            t[y,x] = 255 if char == ' ' else 0
    return t


print(toEdit(toTemplate(digits[8])))

In [ ]:
template = toTemplate(digits[3])
f = grabFrame(80)
b = isBlue(f)
b = (255 * b).astype(np.uint8)
res = cv2.matchTemplate(b, template, cv2.TM_CCOEFF_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
print(max_val, max_loc)
show(res > 0.7 * max_val)
show(f)

In [ ]:
# author: qzane@live.com
# reference: http://stackoverflow.com/a/23565051
# further reading: http://docs.opencv.org/master/da/d56/group__text__detect.html#gsc.tab=0
def text_detect(img,ele_size=(8,3)): #
    if len(img.shape)==3:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_sobel = cv2.Sobel(img,cv2.CV_8U,1,0)#same as default,None,3,1,0,cv2.BORDER_DEFAULT)
    img_threshold = cv2.threshold(img_sobel,0,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY)
    element = cv2.getStructuringElement(cv2.MORPH_RECT,ele_size)
    img_threshold = cv2.morphologyEx(img_threshold[1],cv2.MORPH_CLOSE,element)
    contours = cv2.findContours(img_threshold,0,1)
    Rect = [cv2.boundingRect(i) for i in contours[1] ]#if i.shape[0]>100]
    RectP = [(int(i[0]-i[2]*0.08),int(i[1]-i[3]*0.08),int(i[0]+i[2]*1.1),int(i[1]+i[3]*1.1)) for i in Rect]
    return RectP